In [1]:
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from data_storage import create_connection
from stockstats import StockDataFrame
from risk_metrics import Risk_Metrics
import os
import logging

In [2]:
connection = create_connection("../database/crypto_billionairs.db")

In [3]:
headers = ["time", "open", "high", "low", "close", "volume"]

In [4]:
def momentum(df, lag):
    return df.pct_change(periods=lag)

In [5]:
def prepare_files(path, db_connection, header_list):
    
    for file in os.listdir(path):
        
        try:
            
            if file[-3:] == 'txt':
                df = pd.read_csv(f'./{path}/{file}', names = header_list)
             
                df["momentum_1week"] = momentum(df["close"], 10080)
    
                df["buy_indicator"] = 0
                df.loc[df["momentum_1week"] > 0.025, 'buy_indicator'] = 1
                df.loc[df["momentum_1week"] < -0.08, 'short_indicator'] = -1
            
                file_name = str(file).replace("-", "_")
                
                df.to_sql(f'{file_name[:-4]}_complete_history_long_short', con=db_connection, if_exists="replace", index=False)
                #os.remove(f'{path}/{file}')
        except:
            print("Oh no!")

In [6]:
#prepare_files("../database/", connection, headers)

In [7]:
class Momentum_1m_long_short(Strategy):
   
    
    def init(self):
        # compute the rsi and stochastic oscillator with stockstats and return the buy signal of the current row
        
        self.buy_init = self.I(init_buy_signal, self.data.df)
        self.short_init = self.I(init_short_signal, self.data.df)
        self.data.df.drop(self.data.df.columns.difference(['Open', 'High', 'Low', 'Close', "Volume"]), 1, inplace=True)
        
       
    
    def next(self):
        
        if self.buy_init[-1] == 1 and self.position.is_long is False:
            self.position.close()
            self.buy()

             
        elif self.short_init[-1] == -1 and self.position.is_short is False:
            self.position.close()
            self.sell()

In [8]:
def init_buy_signal(trash):
    return df_temp['buy_indicator']

def init_short_signal(trash):
    return df_temp['short_indicator']

In [9]:
def run_backtesting(db_connection):
    
    global df_temp
    
    table_names = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;", db_connection)
    
    table_names_list = table_names['name'].tolist()

    filtered_table_names = [name for name in table_names_list if "1min_complete_history_long_short" in name and 'trades' not in name]
    
    df_risk = pd.DataFrame(columns = range(8))
    df_risk.columns = ["table_name", "timehorizon", "return", "annualized_return", "sharpe_ratio_annualized", "sortino_ratio_annualized", "maximum_drawdown", "calamr_ratio_annualized"]
    
    for table in filtered_table_names:
        
        df_temp = pd.read_sql_query(f"select * from {table}", db_connection)
        df_backtesting = pd.DataFrame()
        df_backtesting[['Open', 'High', 'Low', 'Close', "Volume"]] = df_temp[['open', 'high', 'low', 'close', 'volume']]
        
        
        bt = Backtest(df_backtesting, Momentum_1m_long_short, cash=100_000, commission=.001)
        stats = bt.run()
        trades = pd.DataFrame(stats['_trades'])
        trades.to_sql(f"trades_{table}", db_connection, if_exists="replace")
        
        risk = Risk_Metrics(trades, df_temp, 0)
        risk_metrics_list = [f"{table}", len(df_backtesting), stats["Return [%]"], risk._annualize(stats["Return [%]"]),
                             risk.sharpe_ratio("momentum_1week"), risk.sortino_ratio("momentum_1week"),
                             risk.max_drawdown(), risk.calmar_ratio(stats["Return [%]"])]
        print(risk_metrics_list)
        #df_risk.append(risk_metrics_list)
        df_risk.loc[len(df_risk)] = risk_metrics_list
        
    
    df_risk.to_sql("cryptocurrencies_risk_metrics_1m", db_connection, if_exists="replace")

In [10]:
run_backtesting(connection)

<ipython-input-9-5d7e79eae585>:21: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df_backtesting, Momentum_1m_long_short, cash=100_000, commission=.001)


['ADA_1min_complete_history_long_short', 1547195, -100.0, -33.971154250110686, -0.1961265415496819, -1.15106784816102, 1.2457557893281659, -27.269513448082208]
                             table_name timehorizon  return  \
0  ADA_1min_complete_history_long_short     1547195  -100.0   

   annualized_return  sharpe_ratio_annualized  sortino_ratio_annualized  \
0         -33.971154                -0.196127                 -1.151068   

   maximum_drawdown  calamr_ratio_annualized  
0          1.245756               -27.269513  
